In [ ]:
# %cd /content
# !apt-get -y install -qq aria2

# !git clone -b v2.5 https://github.com/camenduru/text-generation-webui
# %cd /content/text-generation-webui
# !pip install -q -r requirements.txt

# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/resolve/main/model-00001-of-00002.safetensors -d /content/text-generation-webui/models/Llama-2-7b-chat-hf -o model-00001-of-00002.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/resolve/main/model-00002-of-00002.safetensors -d /content/text-generation-webui/models/Llama-2-7b-chat-hf -o model-00002-of-00002.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/model.safetensors.index.json -d /content/text-generation-webui/models/Llama-2-7b-chat-hf -o model.safetensors.index.json
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/special_tokens_map.json -d /content/text-generation-webui/models/Llama-2-7b-chat-hf -o special_tokens_map.json
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/resolve/main/tokenizer.model -d /content/text-generation-webui/models/Llama-2-7b-chat-hf -o tokenizer.model
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/tokenizer_config.json -d /content/text-generation-webui/models/Llama-2-7b-chat-hf -o tokenizer_config.json
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/config.json -d /content/text-generation-webui/models/Llama-2-7b-chat-hf -o config.json
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/generation_config.json -d /content/text-generation-webui/models/Llama-2-7b-chat-hf -o generation_config.json

# !echo "dark_theme: true" > /content/settings.yaml
# !echo "chat_style: wpp" >> /content/settings.yaml

# %cd /content/text-generation-webui
# !python server.py --share --settings /content/settings.yaml --model /content/text-generation-webui/models/Llama-2-7b-chat-hf

In [ ]:
# !hugging_hub login
# !export HUGGINFACE_KEY_API=""

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install virtualenv
!pip install transformers
!pip install sentencepiece
!pip install langchain
!pip install sentence_transformers
!pip install bitsandbytes-cuda110
!pip install accelerate
!pip install bitsandbytes
# !pip3 install torch torchvision torchaudio
# !pip install packaging
# !pip install flash-attn --no-build-isolation

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, AutoModel, BitsAndBytesConfig, TextStreamer
import torch
import sys
from torch import cuda, bfloat16
import transformers
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate, StringPromptTemplate
from langchain.chains import LLMChain, create_extraction_chain
# sys.path.append("/content/drive/MyDrive/CoLab/Llama2/bin/python3.10/site-packages")

In [ ]:
# very important part, we quantilize Llama model by these codes accerlate
quant_config=BitsAndBytesConfig(
    load_in_4bit=True,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
import accelerate
import bitsandbytes

tokenizer=AutoTokenizer.from_pretrained("daryl149/llama-2-7b-chat-hf")  # daryl149/llama-2-7b-chat-hf
model=LlamaForCausalLM.from_pretrained("daryl149/llama-2-7b-chat-hf",quantization_config=quant_config,device_map="auto",)
# model=AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/CoLab/LLama7b", trust_remote_code=False, torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# !mkdir "/content/drive/MyDrive/CoLab/LLama7b"
# tokenzier.save_pretrained("/content/drive/MyDrive/CoLab/LLama7b")
# model.save_pretrained("/content/drive/MyDrive/CoLab/LLama7b")

In [ ]:
# load on text splitter and initialize text loading
# txt document contain all abstract comes from "risk", doctor, you can put txt file at the same folder
# with this ipynb file
loader=TextLoader(r"risk_abstract.txt")
text=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=4050, chunk_overlap=20)
docs=text_splitter.split_documents(text)

### Single calling upon model; use function generate. We can skip this

In [ ]:
# test_docs=[text.page_content for text in docs[:11]]
query="promopt under construction."#risk_template(test_docs[0])
inputs_ids=tokenizer(query, return_tensors="pt").input_ids.to("cuda")

In [ ]:
len(inputs_ids[0])

16

In [ ]:
generate_result=model.generate(inputs_ids, max_length=1000)
print(tokenizer.decode(generate_result[0], skip_special_tokens=True))

In [ ]:
# this is another embedding method, dont know whether comptable with current docs format
# embed_tokenizer=AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
# embed_model=AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

### Loading Transformer, not appliable so far, please skip

In [ ]:
model_trans="sentence-transformers/all-MiniLM-L6-v2"
model_kwargs={"device": 0}
embedding=HuggingFaceEmbeddings(model_name=model_trans, model_kwargs=model_kwargs)

In [ ]:
docs_extend=[val.page_content for val in docs]

In [ ]:
#---------new added line to test text and embedding function comptability--------------
# print(type(docs), type(text[0].page_content))
# query_result=embedding.embed_documents(docs)
# len(query_result)
# tokenzier.convert_ids_to_tokens([9427])

Do we need a Promopt template? Yes we need



In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

device=f"cuda: {cuda.current_device()}"
!nvidia-smi

Mon Oct  2 07:43:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    28W /  70W |   4981MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
prefix_llm=transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,
    task="text-generation",# temperature=0,
    top_p=0.15, top_k=15,
    max_new_tokens=1060, repetition_penalty=1.2, do_sample=True
)

In [ ]:
llm=HuggingFacePipeline(pipeline=prefix_llm, cache=False) # , model_kwargs={'temperature':0}

In [ ]:
testing=llm.agenerate(hugginface_prompt[:2])
print(testing)

In [ ]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""



def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

"""
a paradigm for direct huggingface transformer loading usage
what about agenerate?
"""
def generate(text):
    prompt = get_prompt(text)
    with torch.autocast('cuda', dtype=torch.bfloat16):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        outputs = model.generate(**inputs,
                                 max_new_tokens=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '</s>')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs#, outputs

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text


In [ ]:
import asyncio
instruction = """help me to identify and extract risks in the given sentence "{words}"? please mark it out and specify the type.
    Also pick keywords to label what ENTIRE sentence talked about from given tags:
    [environment science, environment studies, engineering civil, engineering industrial, management, engineering manufacturing, constructing building, constructing technology, transportation, finance].
    If you cannot judge the organizations or there is no matched tag to label the entire sentence, please return 'None'. Show risks, type of risks, keyword and tags appeared in sentence separately.

    Answer example: "
    risks: \n

    risk_type : \n

    keywords : \n

    tags : \n
    Attention, you can only predict the output in above given criteria and no more than 40 words.No need to specific mark out on every sentence
    """
system_prompt = "You are an expert and summarization and expressing key ideas succintly"

template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["words"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

[INST]<<SYS>>
You are an expert and summarization and expressing key ideas succintly
<</SYS>>

help me to identify and extract risks in the given sentence "{words}"? please mark it out and specify the type.
    Also pick keywords to label what ENTIRE sentence talked about from given tags:
    [environment science, environment studies, engineering civil, engineering industrial, management, engineering manufacturing, constructing building, constructing technology, transportation, finance].
    If you cannot judge the organizations or there is no matched tag to label the entire sentence, please return 'None'. Show risks, type of risks, keyword and tags appeared in sentence separately.

    Answer example: "
    risks: 


    risk_type : 


    keywords : 


    tags : 

    Attention, you can only predict the output in above given criteria and no more than 40 words.No need to specific mark out on every sentence
    [/INST]


In [ ]:
kwarky_list=[]
testing_dict=[val.page_content for val in docs[:6]]
text=docs[142].page_content
output = llm_chain.run(text)
print(output)

  Sure! Here are the risks, types of risks, keywords, and tags extracted from the given sentence:
Risks:
* Heavy snow disaster
* Chemical risk
* Environmental risk

Type of Risks:
* Natural disaster risk
* Ecological risk

Keywords:
* Risk assessment
* Management
* Construction projects
* Technology
* Transportation
* Finance

Tags:
* Environmental science
* Environmental studies
* Civil engineering
* Industrial engineering
* Building construction
* Manufacturing
* Finance


In [ ]:
async def async_generate(achain, context):
    resp = await achain.acall(context)
    print(resp)
    return resp

async def resp_feedback():
  tasks = [async_generate(llm_chain, context) for context in testing_dict]
  await asyncio.gather(*tasks)

In [ ]:
# Ignore Async method provided by LangChain, it now not support Llama2
# reference link: https://github.com/langchain-ai/langchain/issues/11325
await resp_feedback()

NotImplementedError: ignored

### Load_qa_chain method to output
#### (not applicable, bug unidentified, code will be run but output is summzrized prompt)

In [ ]:
chain=load_qa_chain(llm=llm, chain_type="stuff")
res_test=chain.run({"words": docs[4].page_content,
                    "question": query,
                    "input_documents": [docs[4]]})

In [ ]:
total_len=sum([len(docs[i].page_content) for i in range(len(docs))])

In [ ]:
print(res_test)


Risks: construction cost overrun, ship collision and grounding
risk_type: financial, technical
keywords: risk management
tags: environmental science, environmental studies, finance


### create_extraction_chain to output 
#### (not applicable, code will be run but no output, should be oversized input token)

In [ ]:
schema={
    "properties": {
        "risks": {
            "type": "string",
            "description": "any unexpected event that can affect your project. e.g.,Construction cost overrun, Ship collision and grounding"
        },
        "risk_type": {
            "type": "string",
            "description": "the area risk belong to. e.g. Financial, Technical "
        },
        "keywords":{
            "type": "string"
        },
        "tags": {
            "type": "string",
            "description": "select most realted words from given list: [environment science, environment studies, engineering civil, engineering industrial, management, engineering manufacturing, constructing building, constructing technology, transportation, finance]"
        }
    },
    "required": ["risks", "keywords", "tags"]
}
sum_chain=create_extraction_chain(schema, llm, prompt=query)

In [ ]:
sum_chain.run("this is an emergence project")